In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
%cd '/content/drive/MyDrive/zombie'

/content/drive/MyDrive/zombie


In [44]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [45]:
#Load dataset
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')
mobile_plan = pd.read_csv('mobile_plans.csv')
# sample_submission = pd.read_csv('sample_submission.csv')
# train_df

In [46]:
train_df["gender"].fillna(np.random.choice(train_df['gender'][~train_df['gender'].isna()]),inplace = True)
train_df["device_type"].fillna(np.random.choice(train_df['device_type'][~train_df['device_type'].isna()]),inplace = True)
train_df["device_category"].fillna(np.random.choice(train_df['device_category'][~train_df['device_category'].isna()]),inplace = True)
train_df["age_group"].fillna(np.random.choice(train_df['age_group'][~train_df['age_group'].isna()]),inplace = True)
train_df["add_on_tot_rental"].fillna(np.random.choice(train_df['add_on_tot_rental'][~train_df['add_on_tot_rental'].isna()]),inplace = True)
train_df["add_on_count"].fillna(np.random.choice(train_df['add_on_count'][~train_df['add_on_count'].isna()]),inplace = True)
# train_df
test_df["gender"].fillna(np.random.choice(test_df['gender'][~test_df['gender'].isna()]),inplace = True)
test_df["device_type"].fillna(np.random.choice(test_df['device_type'][~test_df['device_type'].isna()]),inplace = True)
test_df["device_category"].fillna(np.random.choice(test_df['device_category'][~test_df['device_category'].isna()]),inplace = True)
test_df["age_group"].fillna(np.random.choice(test_df['age_group'][~test_df['age_group'].isna()]),inplace = True)
test_df["add_on_tot_rental"].fillna(np.random.choice(test_df['add_on_tot_rental'][~test_df['add_on_tot_rental'].isna()]),inplace = True)
test_df["add_on_count"].fillna(np.random.choice(test_df['add_on_count'][~test_df['add_on_count'].isna()]),inplace = True)
# train_df

In [47]:
#Multilabel encoding

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [48]:
train_df = MultiColumnLabelEncoder(columns = ['next_month_plan', 'device_type', 'device_category', 'gender','district_name', 'age_group']).fit_transform(train_df)
test_df = MultiColumnLabelEncoder(columns = ['device_type', 'device_category', 'gender','district_name', 'age_group']).fit_transform(test_df)

In [38]:
train_df[train_df['average_monthly_bill_amount']<0] =0
test_df[test_df['average_monthly_bill_amount']<0] =0

In [39]:
#train test split keeping 25% of the data for testing
feature_cols = ['device_type', 'device_category', 'gender', 'district_name','age_group','network_stay','average_monthly_bill_amount','dusage_sum','vusage_offnet_sum','vusage_onnet_sum']
X = train_df[feature_cols]
y = train_df.next_month_plan
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [40]:
#Using MLPClassifier
sc = StandardScaler()
scaler = sc.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
mlp = MLPClassifier()
mlp = MLPClassifier(max_iter=1000, hidden_layer_sizes=(100,50,30)).fit(X_train, y_train)
y_pred=mlp.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.464


In [41]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(mlp, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

y_data = test_df[feature_cols]
y_valid = loaded_model.predict(y_data)

my_submission = pd.DataFrame({'primary_identifier': test_df.primary_identifier, 'next_month_plan': y_valid})
my_submission["next_month_plan"].replace({0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8}, inplace=True)

In [42]:
my_submission.to_csv('submissionv2.csv', index=False)